In [4]:
import sys  
import os

# some basic libraries 
from platform import python_version

import htsat_config
from htsat import HTSAT_Swin_Transformer 

# print system information
print('Python Version     : ', python_version())

Python Version     :  3.9.13


In [5]:
# TODO - data pipeline in pytorch lightening

In [6]:
# Create an instance of the HTSAT model
model = HTSAT_Swin_Transformer(
    spec_size=htsat_config.htsat_spec_size,
    patch_size=htsat_config.htsat_patch_size,
    in_chans=1,
    num_classes=htsat_config.classes_num,
    window_size=htsat_config.htsat_window_size,
    config = htsat_config,
    depths = htsat_config.htsat_depth,
    embed_dim = htsat_config.htsat_dim,
    patch_stride=htsat_config.htsat_stride,
    num_heads=htsat_config.htsat_num_head)

AttributeError: module 'htsat_config' has no attribute 'mel_bins'

In [ ]:
# show model details
model